[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/sdxl-colab/blob/main/sdxl_v1.0_webui_dev_colab.ipynb)

In [ ]:
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1
!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
!apt -y install -qq aria2
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!git clone -b v2.6 https://dagshub.com/camenduru/ui
!git clone https://dagshub.com/camenduru/control /content/ui/extensions/control
!git clone https://dagshub.com/camenduru/deforum /content/ui/extensions/deforum
!git clone https://github.com/camenduru/tunnels /content/ui/extensions/tunnels
!git clone https://github.com/camenduru/sd-civitai-browser /content/ui/extensions/sd-civitai-browser
!git clone https://github.com/etherealxx/batchlinks-webui /content/ui/extensions/batchlinks-webui
%cd /content/ui
!git reset --hard
!git -C /content/ui/extensions/deforum reset --hard
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared_options.py
!sed -i -e 's/\.sd-webui-controlnet/\.control/g' /content/ui/extensions/deforum/scripts/deforum_helpers/deforum_controlnet.py

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d /content/ui/models/Stable-diffusion -o sd_xl_refiner_1.0.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d /content/ui/models/Stable-diffusion -o sd_xl_base_1.0.safetensors

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple